<a href="https://colab.research.google.com/github/Keren-Neeq/DS4A-Team1/blob/main/notebooks/Team1_ExtendedEDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How Remote Work Is Transforming America's Geographical Landscape
Team #1 | Ashek Ag Mohamed, Caprina Beal, Nzinga Eduardo Rodriguez, Keren Osabutey, Erika Smith

## Business Problem
### Overview

One of the most significant impacts of the Covid-19 pandemic may be a shift in how the workplace and workforce operate in the future. According to Forbes, approximately 5% of white-collar employees in the U.S. worked primarily from home before the outbreak. This figure skyrocketed to 71% during the pandemic. If given a choice to continue working from home post-pandemic, 54% of these workers indicated they would continue to do so, according to Pew Research Center. These figures will likely vary across occupations, industries, income, and education levels. As remote work becomes more popular, white-collar workers are likely to relocate from city centers to suburban areas, accelerating and changing geographic trends.

A report by the Economic Innovation Group (EIG) indicated that more than two-thirds of large urban counties saw their populations decline. Marking the first time in 50 years, counties with a population over 250,000  experienced negative growth. Although migration patterns had been in effect before Covid-19, the explosion of remote work during the pandemic, reductions in births, rising deaths, and delayed immigration accelerated the shift. EIG indicated other contributing factors, including high median home and apartment rental prices, low housing inventory, and increasing costs of consumer goods such as gas. 

Using data curated from Glassdoor, Zillow, the U.S. Department of Housing, the U.S. Bureau of Labor Statistics, and the U.S. Census Bureau, our analysis will answer the following:


1.   What are the top 10 cities in the U.S. with the most remote work opportunities?
2.   What are the top 10 states with the most remote work opportunities?
3.   Which jobs are likely to experience more significant increases in remote work?
4.   Which workers are most likely to have the option to work from home?
5.   How have city growth rates changed, and how is the population shifting?
6.   How have the average annual expenditures and population characteristics changed?
7.   How have the average single family home, and apartment rent prices changed over time?
8.   What are the real estate implications of an increase in remote work?
9.   Is there a correlation between inflation, home prices, and rent prices?
10. Has the inflation rate affected housing inventory? 
11. How has the cost of consumer goods changed?
12. What changes have occurred over time for consumer spending? 


## Import Libraries and Data

In [ ]:
pip install jupyter-dash

In [4]:
import os
import re
import io
import sys
import warnings

import numpy as np
import pandas as pd
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
import plotly.express as px

from functools import reduce 
from google.colab import files
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output

sys.path.insert(0, "../utils")
warnings.filterwarnings('ignore')
sns.set(context='talk',style = 'whitegrid')
%matplotlib inline

In [5]:
# Upload File From Computer
uploaded = files.upload()

Saving hud_yearly.csv to hud_yearly.csv


In [6]:
# Department of Housing Avg. Rent Prices
hud_df = pd.read_csv(io.BytesIO(uploaded['hud_yearly.csv']))

### 1.   What are the top 10 cities in the U.S. with the most remote work opportunities?

### 2.   What are the top 10 states with the most remote work opportunities?

### 3.   Which jobs are likely to experience more significant increases in remote work?

### 4.   Which workers are most likely to have the option to work from home?

### 5.   How have city growth rates changed, and how is the population shifting?

### 6.   How have the average annual expenditures and population characteristics changed?

### 7.   How have the average single family home, and apartment rent prices changed over time?

In [8]:
hud_df

,City,State,Bedrooms,Year,Avg Rent,Change,YoY,2Y Change,Yo2Y,3Y Change,Yo3Y,4Y Change,Yo4Y
0,Phoenix,AZ,studio,2019,847,103,13.84,NaN,NaN,NaN,NaN,NaN,NaN
1,Los Angeles,CA,studio,2019,1279,121,10.45,NaN,NaN,NaN,NaN,NaN,NaN
2,Sacramento,CA,studio,2019,952,99,11.61,NaN,NaN,NaN,NaN,NaN,NaN
3,San Francisco,CA,studio,2019,2197,375,20.58,NaN,NaN,NaN,NaN,NaN,NaN
4,Chicago,IL,studio,2019,956,41,4.48,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Boston,MA,4,2022,3540,287,8.82,372.0,11.74,409.0,13.06,969.0,37.69
196,New York,NY,4,2022,3316,143,4.51,532.0,19.11,673.0,25.46,841.0,33.98
197,Nashville,TN,4,2022,2173,198,10.03,243.0,12.59,351.0,19.26,435.0,25.03
198,Austin,TX,4,2022,2416,222,10.12,209.0,9.47,288.0,13.53,317.0,15.10


In [10]:
hud22_df = hud_df.query("Year==2022")

In [13]:
hud22_df.reset_index(drop=True, inplace=True)

In [15]:
# Convert numbers from float to int
hud22_df['2Y Change'] = hud22_df['2Y Change'].astype('int')
hud22_df['3Y Change'] = hud22_df['3Y Change'].astype('int')
hud22_df['4Y Change'] = hud22_df['4Y Change'].astype('int')

In [16]:
hud22_df.head()

,City,State,Bedrooms,Year,Avg Rent,Change,YoY,2Y Change,Yo2Y,3Y Change,Yo3Y,4Y Change,Yo4Y
0,Phoenix,AZ,studio,2022,1344,339,33.73,411,44.05,497,58.68,600,80.65
1,Los Angeles,CA,studio,2022,1534,150,10.84,165,12.05,255,19.94,376,32.47
2,Sacramento,CA,studio,2022,1277,169,15.25,217,20.47,325,34.14,424,49.71
3,San Francisco,CA,studio,2022,2156,41,1.94,-194,-8.26,-41,-1.87,334,18.33
4,Chicago,IL,studio,2022,1158,99,9.35,146,14.43,202,21.13,243,26.56


In [42]:
fig = px.bar(hud22_df, 
             x="YoY", 
             y="City", 
             hover_data={'Avg Rent': ':$,.0f', 
                         'Change': ':$,.0f', 
                         'YoY': ':.2f%'
                         },
             title="% Change in Apartment Rent Prices, 2022-2021 ", 
             barmode="group",
             color="Bedrooms", 
             labels={'YoY': 'YoY (%)'},
             height=700,
             orientation='h')

fig.update_layout(
    hoverlabel=dict(
        font_size=16,
        font_family="Rockwell"
    )
)

fig.show()

### 8.   What are the real estate implications of an increase in remote work?

### 9.   Is there a correlation between inflation, home prices, and rent prices?

### 10.   Has the inflation rate affected housing inventory? 

### 11.   How has the cost of consumer goods changed?

### 12.   What changes have occurred over time for consumer spending? 